In [2]:
import os


In [3]:
%pwd


'/Users/ericliao/Desktop/projects/MLOps-chicken-disease-classifier/research'

In [4]:
os.chdir("../")


In [5]:
%pwd


'/Users/ericliao/Desktop/projects/MLOps-chicken-disease-classifier'

In [6]:
import tensorflow as tf


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
from urllib.parse import urlparse


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [13]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-09-29 08:57:33,314: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-29 08:57:33,316: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-29 08:57:33,316: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 8s 908ms/step - loss: 0.4627 - accuracy: 0.8793
[2023-09-29 08:57:41,043: INFO: common: json file saved at: scores.json]


TypeError: isinstance() arg 2 must be a type or tuple of types

In [11]:
from ensure import ensure_annotations
from pathlib import Path
import json
from typing import Dict


In [18]:
# @ensure_annotations
def save_json(path: Path, data) -> None:
    """save json data

    Args:
        path (Path): path to json file
        data (dict): data to be saved in json file
    """
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

    print(f"json file saved at: {path}")

In [19]:
scores = {"loss": 0.20, "accuracy": 0.89}

save_json(path=Path("scores_testing.json"), data=dict(scores))

json file saved at: scores_testing.json


In [8]:
type(scores)

dict

In [14]:
dict(scores)

{'loss': 0.2, 'accuracy': 0.89}